<a href="https://colab.research.google.com/github/duper203/MLP_pytorch/blob/main/three-layer/variable_learningRate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1 : MLP Classification (Pytorch)


In [ ]:
name = input("Name :")
ID = input("student ID :")

Name :김혜수
student ID :2110883


## 실습
___


## Matrix 미분 정리
$H = XW+b$</br>
$L = f(H)$</br>
$\frac{\partial L}{\partial W} = \frac{\partial H}{\partial W} \times \frac{\partial L}{\partial H} = \frac{\partial (XW+b)}{\partial W} \times\frac{\partial L}{\partial H} = X *\frac{\partial L}{\partial H}$</br>
$\frac{\partial L}{\partial X} = \frac{\partial L}{\partial H} \times \frac{\partial H}{\partial X} = \frac{\partial L}{\partial H} \times \frac{\partial (XW+b)}{\partial X} = \frac{\partial L}{\partial H}*W$</br>
$\frac{\partial L}{\partial b}=1*\frac{\partial L}{\partial H}$</br>

3 Layer Network를 완성하시오


In [1]:
import torch

class ThreeLayerNet(torch.nn.Module):
  def __init__(self, input_size, hidden_size_one,hidden_size_two,output_size, std=1e-4):
    super(ThreeLayerNet, self).__init__()
    torch.manual_seed(0)

    #---------과제---------#
    #네트워크를 설계해보세요#
    #torch.nn.Linear가 MLP 한 층을 나타냅니다#
    #Activation function도 잊지 마세요#
    #! hidden size는 자유롭게 설계해보세요 !#
    #---------과제---------#

    self.layer1=torch.nn.Linear(input_size, hidden_size_one)
    self.layer2=torch.nn.Linear(hidden_size_one, hidden_size_two)
    self.layer3=torch.nn.Linear(hidden_size_two, output_size)
    self.relu=torch.nn.ReLU()


  def forward(self, X, y = None):
    #---------과제---------#
    #feedfoward 부분을 코딩해보세요#
    #천천히 순서대로 작성하시면 됩니다.#
    #마지막에 softmax 잊지 마세요#
    #최종 결과물 변수의 이름은 output으로 해주세요#
    #---------과제---------#
    feature1=self.relu(self.layer1(X))
    feature2=self.relu(self.layer2(feature1))
    feature3=self.relu(self.layer3(feature2))

    output=torch.nn.functional.log_softmax(feature3, dim=1)

    return output

  def accuracy(self, X, y):
      p = self.forward(X)
      pred = torch.argmax(p,dim=1)
      acc = torch.sum(pred==y).float()/X.shape[0]*100

      return acc


#### Data

In [2]:
from keras.datasets import mnist
import numpy as np
from matplotlib import pyplot as plt
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [3]:
print("shape of x_train :",x_train.shape)
print("shape of y_train :",y_train.shape)
print("shape of x_test :",x_test.shape)
print("shape of y_test :",y_test.shape)

shape of x_train : (60000, 28, 28)
shape of y_train : (60000,)
shape of x_test : (10000, 28, 28)
shape of y_test : (10000,)


In [4]:
def preprocessing_data(x_train, x_test, y_train, y_test):
    x_train = x_train[:5000]
    y_train = y_train[:5000]
    x_test = x_test[:1000]
    y_test = y_test[:1000]

    #change dtype
    x_train = torch.from_numpy(x_train).cuda()
    x_test = torch.from_numpy(x_test).cuda()

    x_train = torch.tensor(x_train,dtype=torch.float64, device="cuda:0")
    y_train = torch.tensor(y_train,device="cuda:0")

    x_test = torch.tensor(x_test,dtype=torch.float64,device="cuda:0")
    y_test = torch.tensor(y_test,device="cuda:0")

    #reshaping : 2D -> 1D
    x_train = x_train.reshape([x_train.shape[0],-1])
    x_test = x_test.reshape(x_test.shape[0],-1)


    #normalization : 값의 정규화
    mean_value = torch.mean(x_train, dim=0)
    x_train -= mean_value
    x_train = x_train/255
    x_test -= mean_value
    x_test = x_test/255


    return x_train, x_test, y_train, y_test

In [5]:
x_train, x_test, y_train, y_test = preprocessing_data(x_train, x_test, y_train, y_test)


<ipython-input-4-8ebaba95c783>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_train = torch.tensor(x_train,dtype=torch.float64, device="cuda:0")
<ipython-input-4-8ebaba95c783>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test = torch.tensor(x_test,dtype=torch.float64,device="cuda:0")


In [6]:
print("shape of x_train :",x_train.shape)
print("shape of y_train :",y_train.shape)
print("shape of x_test :",x_test.shape)
print("shape of y_test :",y_test.shape)

shape of x_train : torch.Size([5000, 784])
shape of y_train : torch.Size([5000])
shape of x_test : torch.Size([1000, 784])
shape of y_test : torch.Size([1000])


####Training

In [7]:
N = x_train.shape[0]
input_size = x_train.shape[1]

#---------과제---------#

## 고정 변수
output_size = 10 # output target 종류의 개수 mnist 데이터 클래스 10개 (고정)

epoch_size = 10 # 학습 수
hidden_size_one= 128
hidden_size_two= 64

## 실험 할 값
learning_rates = [0.1, 0.01, 0.001, 0.0001, 0.00001]
results_lr = {}

#---------과제---------#

for learning_rate in learning_rates:

  Network = ThreeLayerNet(input_size=input_size, hidden_size_one=hidden_size_one,hidden_size_two=hidden_size_two,output_size=output_size)
  Network = Network.cuda()
  history = {'val_acc': [],'val_loss': []}

  #loss 와 optimzer 선언
  optimizer = torch.optim.SGD(Network.parameters(), lr=learning_rate)
  criterion = torch.nn.CrossEntropyLoss()

  #코드를 보며 epoch, batch에 대해서 이해해봅시다.
  for i in range(epoch_size+1):
      for j in range(N):

          optimizer.zero_grad()

          x_now = x_train[j].unsqueeze(0).type(torch.cuda.FloatTensor)
          y_now = y_train[j].unsqueeze(0)

          y_predict = Network(x_now)

          #---------과제---------#
          #loss 계산 후 weight update#
          #---------과제---------#

          # loss 계산
          loss=criterion(y_predict,y_now)

          # BP
          loss.backward()

          # weight update := optimizer
          optimizer.step()

      #accuracy와 loss를 기록해둡시다.
      val_acc = 0
      val_loss = 0
      for k in range(x_test.shape[0]):
          val_acc += Network.accuracy(x_test[k].unsqueeze(0).type(torch.cuda.FloatTensor), y_test[k])
          val_loss += criterion((Network.forward(x_test[k].unsqueeze(0).type(torch.cuda.FloatTensor))), y_test[k].unsqueeze(0)).detach()
      history["val_acc"].append((val_acc/x_test.shape[0]).cpu())
      history["val_loss"].append((val_loss/x_test.shape[0]).cpu())

      if i % 10 == 0:
          print("============================================")
          print("learning_rate:",learning_rate)
          print(i, "train accuracy :", Network.accuracy(x_train.type(torch.cuda.FloatTensor), y_train))
          print(i, "train loss     :", loss)
          print(i, "test accuracy :", val_acc/x_test.shape[0])
          print(i, "test loss     :", val_loss/x_test.shape[0])
          print("============================================")

      results_lr[learning_rate] = {'final_acc': history['val_acc'][-1], 'final_loss': history['val_loss'][-1], 'history': history}


learning_rate: 0.1
0 train accuracy : tensor(18.3200, device='cuda:0')
0 train loss     : tensor(2.3026, device='cuda:0', grad_fn=<NllLossBackward0>)
0 test accuracy : tensor(16.4000, device='cuda:0')
0 test loss     : tensor(2.1676, device='cuda:0')
learning_rate: 0.1
10 train accuracy : tensor(17.2200, device='cuda:0')
10 train loss     : tensor(2.3026, device='cuda:0', grad_fn=<NllLossBackward0>)
10 test accuracy : tensor(16.6000, device='cuda:0')
10 test loss     : tensor(2.0809, device='cuda:0')
learning_rate: 0.01
0 train accuracy : tensor(56.3400, device='cuda:0')
0 train loss     : tensor(0.1247, device='cuda:0', grad_fn=<NllLossBackward0>)
0 test accuracy : tensor(55.5000, device='cuda:0')
0 test loss     : tensor(1.2205, device='cuda:0')
learning_rate: 0.01
10 train accuracy : tensor(69.3800, device='cuda:0')
10 train loss     : tensor(0.0002, device='cuda:0', grad_fn=<NllLossBackward0>)
10 test accuracy : tensor(65.8000, device='cuda:0')
10 test loss     : tensor(0.8939, dev

In [8]:
results_lr

{0.1: {'final_acc': tensor(16.6000),
  'final_loss': tensor(2.0809),
  'history': {'val_acc': [tensor(16.4000),
    tensor(25.5000),
    tensor(20.9000),
    tensor(18.3000),
    tensor(17.7000),
    tensor(17.8000),
    tensor(15.3000),
    tensor(16.9000),
    tensor(17.9000),
    tensor(14.6000),
    tensor(16.6000)],
   'val_loss': [tensor(2.1676),
    tensor(1.9832),
    tensor(2.0797),
    tensor(2.0678),
    tensor(2.1058),
    tensor(2.1038),
    tensor(2.1003),
    tensor(2.1267),
    tensor(2.1150),
    tensor(2.1061),
    tensor(2.0809)]}},
 0.01: {'final_acc': tensor(65.8000),
  'final_loss': tensor(0.8939),
  'history': {'val_acc': [tensor(55.5000),
    tensor(55.9000),
    tensor(56.5000),
    tensor(64.1000),
    tensor(65.1000),
    tensor(65.7000),
    tensor(64.7000),
    tensor(65.7000),
    tensor(65.5000),
    tensor(65.5000),
    tensor(65.8000)],
   'val_loss': [tensor(1.2205),
    tensor(1.1286),
    tensor(1.0979),
    tensor(1.0426),
    tensor(0.9227),
    te